In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"
new_test = "./new_val_in/new_val_in"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):
        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
val_testset  = ArgoverseDataset(data_path=new_test)

In [3]:
batch_sz = 4
batch_sz_test = 1

# For Miami Dataset
def my_collate_val_MIA(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inpMIA = []
    outMIA = []
    
    for scene in batch:
        if scene['city'] == 'MIA':
            inpMIA = [numpy.dstack([scene['p_in'], scene['v_in']])]
            outMIA = [numpy.dstack([scene['p_out'], scene['v_out']])]
    
    inpMIA = torch.FloatTensor(inpMIA)
    outMIA = torch.FloatTensor(outMIA)
    return [inpMIA, outMIA]

def my_collate_val_PIT(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inpPIT = []
    outPIT = []
    
    for scene in batch:
        if scene['city'] == 'PIT':
            inpPIT = [numpy.dstack([scene['p_in'], scene['v_in']])]
            outPIT = [numpy.dstack([scene['p_out'], scene['v_out']])]
                
    
    inpPIT = torch.FloatTensor(inpPIT)
    outMIA = torch.FloatTensor(outPIT)
    return [inpPIT, outPIT]

val_loader_MIA = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, 
                            collate_fn=my_collate_val_MIA, num_workers=0)
val_loader_PIT = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, 
                            collate_fn=my_collate_val_PIT, num_workers=0)

In [4]:
# For Miami Dataset
def my_collate_train_MIA(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inpMIA = []
    outMIA = []
    
    for scene in batch:
        if scene['city'] == 'MIA':
            inpMIA = [numpy.dstack([scene['p_in'], scene['v_in']])]
            outMIA = [numpy.dstack([scene['p_out'], scene['v_out']])]
    
    inpMIA = torch.FloatTensor(inpMIA)
    outMIA = torch.FloatTensor(outMIA)
    return [inpMIA, outMIA]

def my_collate_train_PIT(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inpPIT = []
    outPIT = []
    
    for scene in batch:
        if scene['city'] == 'PIT':
            inpPIT = [numpy.dstack([scene['p_in'], scene['v_in']])]
            outPIT = [numpy.dstack([scene['p_out'], scene['v_out']])]
                
    
    inpPIT = torch.FloatTensor(inpPIT)
    outMIA = torch.FloatTensor(outPIT)
    return [inpPIT, outPIT]

test_loader_MIA = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, 
                            collate_fn=my_collate_train_MIA, num_workers=0)
test_loader_PIT = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, 
                            collate_fn=my_collate_train_PIT, num_workers=0)

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class LSTM(nn.Module):
    
    def __init__(self):
        super(LSTM, self).__init__()
        self.hidden_dim = 2000
        self.num_layers=3
        self.lstm = nn.LSTM(240, self.hidden_dim, num_layers=self.num_layers, batch_first = True)
        self.linear1 = nn.Conv1d(self.hidden_dim, 240, 1)

    def forward(self, x):
        x,_ = self.lstm(x)
        x = torch.transpose(x,1,2)
        x = self.linear1(x)
        x = torch.transpose(x,1,2)
        
        return x
    
    def forward_test(self, x, num_steps = 30):
        res=[]
        h = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        c = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        for step in range(num_steps):
            x, (h,c) = self.lstm(x, (h,c))
            x = x[:,-1:]
            x = torch.transpose(x,1,2)
            x = self.linear1(x)
            x = torch.transpose(x,1,2)
            res.append(x)
        res = torch.cat(res,1)
        return res

In [10]:
from tqdm import tqdm_notebook as tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = "cuda:0"
model = LSTM().to(device)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

loss_ema = -1
loss_ema2 = -1

# For MIA
for epoch in range(1):
    for i_batch, sample_batch in enumerate (val_loader_MIA):
        inp,out = sample_batch
        inp = inp.cuda()
        out = out.cuda()
        # mixed = torch.cat([inp,out],2).transpose(1,2).reshape(-1,49,240)
        mixed = torch.cat([inp,out],2)
        print(inp.shape)
        print(out.shape)
        mixed = mixed.transpose(1, 2)
        mixed = mixed.reshape(-1, 49, 240)
        y_pred = model(mixed[:,:-1])[:,-30:]
        y_pred = y_pred.reshape(-1,30,60,4).transpose(1,2)
        loss = (torch.mean((y_pred-out)**2))**0.5
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if loss_ema < 0:
            loss_ema = loss
        loss_ema = loss_ema*0.99 + loss*0.01
        
        with torch.no_grad():
            y_pred2 = model.forward_test(inp.transpose(1,2).reshape(-1,19,240))
            y_pred2 = y_pred2.reshape(-1, 30, 60, 4).transpose(1,2)
            loss2 = torch.mean((y_pred2-out)**2)**0.5
            if loss_ema2 < 0:
                loss_ema2 = loss2
            loss_ema2 = loss_ema2*0.99 + loss*0.01
        
        if i_batch%10 == 0:
            print('loss full', epoch, i_batch, loss_ema.item(), loss.item())
            print('loss test', epoch, i_batch, loss_ema2.item(), loss2.item())
            
torch.save(model.state_dict(),"Simple_LSTM_MIA")

torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
loss full 0 0 902.48291015625 902.48291015625
loss test 0 0 902.315673828125 902.31396484375
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
loss full 0 10 869.6507568359375 388.4565124511719
loss test 0 10 869.4994506835938 388.16656494140625
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30, 4])
torch.Size([1, 60, 19, 4])
torch.Size([1, 60, 30

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
# For PIT
for epoch in range(1):
    for i_batch, sample_batch in enumerate (val_loader_PIT):
        inp,out = sample_batch
        inp = inp.cuda()
        out = out.cuda()
        mixed = torch.cat([inp,out],2).transpose(1,2).reshape(-1,49,240)
        y_pred = model(mixed[:,:-1])[:,-30:]
        y_pred = y_pred.reshape(-1,30,60,4).transpose(1,2)
        loss = (torch.mean((y_pred-out)**2))**0.5
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if loss_ema < 0:
            loss_ema = loss
        loss_ema = loss_ema*0.99 + loss*0.01
        
        with torch.no_grad():
            y_pred2 = model.forward_test(inp.transpose(1,2).reshape(-1,19,240))
            y_pred2 = y_pred2.reshape(-1, 30, 60, 4).transpose(1,2)
            loss2 = torch.mean((y_pred2-out)**2)**0.5
            if loss_ema2 < 0:
                loss_ema2 = loss2
            loss_ema2 = loss_ema2*0.99 + loss*0.01
        
        if i_batch%10 == 0:
            print('loss full', epoch, i_batch, loss_ema.item(), loss.item())
            print('loss test', epoch, i_batch, loss_ema2.item(), loss2.item())
torch.save(model.state_dict(),"Simple_LSTM_PIT")